In [ ]:

from mrjob.job import MRJob
import xml.etree.ElementTree as ET

class ReportQualityMR(MRJob):

    def mapper(self, _, line):
        root = ET.fromstring(line)
        org_name = root.find('Название').text
        report_date = root.find('Отчетная_дата').text
        # Предположим, что качество отчетности оценивается по наличию заполненных полей "Сумма_активов" и "Сумма_обязательств"
        assets = root.find('Сумма_активов').text
        liabilities = root.find('Сумма_обязательств').text

        if assets and liabilities:
            quality_score = 10  
        else:
            quality_score = 1  

        yield org_name, (report_date, quality_score)

    def reducer(self, org_name, values):
        total_score = 0
        count = 0

        for report_date, quality_score in values:
            total_score += quality_score
            count += 1

        avg_quality = total_score / count if count > 0 else 0

        yield org_name, avg_quality

if __name__ == '__main__':
    ReportQualityMR.run()